In [58]:
import pandas as pd
import numpy as np
import os

In [59]:
"""
Identify all the region names in singapore
"""
os.chdir('C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\Rental_Fleet_Demand_Forecast\data\station_names_n_cluster_center_id')
stn_df = pd.read_csv("station_cc_id_60_0.5.csv")
region_names = stn_df["name"].apply( lambda x :  x.split('/')[0] ).unique()
region_names


array(['Ang Mo Kio', 'Bedok', 'Bidadari', 'Bishan', 'Bukit Batok',
       'Bukit Merah', 'Bukit Panjang', 'Bukit Timah', 'Central', 'Changi',
       'Choa Chu Kang', 'Clementi', 'Geylang', 'Hougang', 'Jurong East',
       'Jurong West', 'Kallang', 'Kembangan', 'Marine Parade', 'Novena',
       'Pasir Ris', 'Punggol', 'Queenstown', 'Sembawang', 'Sengkang',
       'Sentosa', 'Serangoon', 'Tampines', 'Toa Payoh', 'Whampoa',
       'Woodlands', 'Woodland', 'Yishun'], dtype=object)

In [60]:
"""
Create region inflow
"""
os.chdir('C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\Rental_Fleet_Demand_Forecast\data\cluster_level\inflow_data')
inf_df = pd.read_csv("cluster_inflow.csv")

region_inflow_df =  pd.DataFrame()

for reg in region_names:
    region_df = stn_df[stn_df["name"].apply( lambda x :  x.split('/')[0] ) == reg]
    cluster_names = np.sort(region_df["cc_id"].unique())
    cluster_names = cluster_names.astype(str)
    start_time_in = inf_df.index[0]
    end_time_in = inf_df.index[-1]
    reg_inf_df = inf_df.loc[:,cluster_names]
    region_inflow_df[reg] = reg_inf_df.sum(axis=1)

os.chdir('C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\Rental_Fleet_Demand_Forecast\data\\region_level\inflow_data')
region_inflow_df.to_csv('region_inflow.csv')

In [61]:
len(region_inflow_df.columns)

33

In [62]:
"""
Create region met outflow
"""

## met outflow(demand)
os.chdir('C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\Rental_Fleet_Demand_Forecast\data\cluster_level\outflow_data')
outf_df = pd.read_csv('cluster_outflow.csv')

region_met_outflow_df =  pd.DataFrame()

for reg in region_names:
    region_df = stn_df[stn_df["name"].apply( lambda x :  x.split('/')[0] ) == reg]
    cluster_names = np.sort(region_df["cc_id"].unique())
    cluster_names = cluster_names.astype(str)
    start_time_in = outf_df.index[0]
    end_time_in = outf_df.index[-1]
    reg_outf_df = outf_df.loc[:,cluster_names]
    region_met_outflow_df[reg] = reg_outf_df.sum(axis=1)


region_met_outflow_df
os.chdir('C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\Rental_Fleet_Demand_Forecast\data\\region_level\outflow_data')
region_met_outflow_df.to_csv('region_met_outflow.csv')



In [63]:
len(region_met_outflow_df.columns)

33

In [64]:
"""
Create region unmet outflow
"""

# convert unmet station outflow to unmet region outflow
os.chdir('C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\Rental_Fleet_Demand_Forecast\data\station_level\outflow_data')
UnmetDemand_df_station_level = pd.read_parquet('UnmetDemand_df_station_level.parquet')
# impute missing stations
for s in [30,36,364,2,188]:
    UnmetDemand_df_station_level[str(s)] = 0

os.chdir('C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\Rental_Fleet_Demand_Forecast\data\station_names_n_cluster_center_id')
stations_df = pd.read_csv('stations.csv')
stations_df

stations_in_region_dict = {}
# identify stations belonging to a region
for reg in region_names:
    stations_in_region_dict[reg] = stations_df[stations_df["name"].apply( lambda x :  x.split('/')[0] ) == reg]['ID'].values.astype(str)

# sum all the stations unmet outflow into a single time series
region_unmet_outflow_df = pd.DataFrame()
for reg in region_names:
    region_unmet_outflow_df[reg] = UnmetDemand_df_station_level.loc[:,stations_in_region_dict[reg]].sum(axis=1)

region_unmet_outflow_df
os.chdir('C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\Rental_Fleet_Demand_Forecast\data\\region_level\outflow_data')
region_unmet_outflow_df.to_csv('region_unmet_outflow.csv')


In [65]:
len(region_unmet_outflow_df.columns)

33